In [1]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn import utils

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [2]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)

In [3]:
def text_to_list(x):
    if pd.isna(x):
        return ''
    else:
        return ast.literal_eval(x)

def parse_json(x):
    try:
        return json.loads(x.replace("'", '"'))[0]['name']
    except:
        return ''
    
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #number of genres we are looking at
        for i in range(numElems):
            genre_str = (json_genres[i]['name'])
            if genre_str in genre_map.keys():
                ret[genre_dict[genre_map[genre_str]]] = 1
        return ret
    except Exception as excep:
        print('Exception' + str(excep))
        return ''

In [4]:
genre_dict = {}
genre_dict['Action-Adventure'] = 0
genre_dict['Romance'] = 1
genre_dict['Horror-Thriller'] = 2
genre_dict['Comedy'] = 3
genre_dict['Science Fiction'] = 4
genre_dict['Drama'] = 5
genre_dict

{'Action-Adventure': 0,
 'Romance': 1,
 'Horror-Thriller': 2,
 'Comedy': 3,
 'Science Fiction': 4,
 'Drama': 5}

In [5]:
genre_map = {}
genre_map['Adventure'] = 'Action-Adventure'
genre_map['Romance'] = 'Romance'
genre_map['Horror'] = 'Horror-Thriller'
genre_map['Thriller'] = 'Horror-Thriller'
genre_map['Comedy'] = 'Comedy'
#genre_map['War'] = 'Action-Adventure'#not sure about this
genre_map['Action'] = 'Action-Adventure'
genre_map['Science Fiction'] = 'Science Fiction'
genre_map['Fantasy'] = 'Science Fiction'
genre_map['Drama'] = 'Drama'

In [6]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret
    return ret

In [7]:
labels_vects = getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [8]:
#put to lower case, remove punctation
def cleanText(text):
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text
all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [9]:
all_data = all_data[all_data.genres_vect.map(sum) > 0]

In [10]:
#logistic regression data
lr_data = all_data[['cleanOverview', 'genres_vect', 'overview']]

In [11]:
train, test = train_test_split(lr_data, test_size=0.2, random_state=42)

CNN STUFF here

In [12]:
#get word embeddings
x = train['cleanOverview'].values.tolist()
y = train['genres_vect']

In [13]:
x_test = test['cleanOverview'].values.tolist()
y_test = test['genres_vect']

In [14]:
y_train = y.tolist()
y_train = np.array(y_train)

In [15]:
y_test = y_test.tolist()
y_test = np.array(y_test)

In [16]:
tok = [word_tokenize(sent) for sent in x]

In [17]:
word_vec_len = 32
model = Word2Vec(tok, min_count = 1, size=word_vec_len)

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words_kept = 100000 #using 100000 most popular words, use throughout

tokenizer = Tokenizer(num_words_kept)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

max_seq_len = 100

x_train_seq = pad_sequences(sequences, maxlen=max_seq_len)

Using TensorFlow backend.


In [19]:
test_sequences = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(test_sequences, maxlen=max_seq_len)

In [20]:
embeddings_index = {}
for w in model.wv.vocab.keys():
    embeddings_index[w] = model.wv[w]


embedding_matrix = np.zeros((num_words_kept, word_vec_len))
for word, i in tokenizer.word_index.items():
    if i >= num_words_kept:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def get_per_label_metrics(real_labels_matrix, predictions_labels_matrix):
    for genre in genre_dict.keys():
        index = genre_dict[genre]
        real_labels_vect = real_labels_matrix[:, index]
        prediction_vect = predictions_labels_matrix[:,index]
        print("Accuruacy for " + genre + ": " + str(accuracy_score(real_labels_vect, prediction_vect)))
        print("Precision for " + genre + ": " + str(precision_score(real_labels_vect, prediction_vect)))
        print("Recall for " + genre + ": " + str(recall_score(real_labels_vect, prediction_vect)))
        print()

In [22]:
#size of intersection of predicted and actual labels divided by size of their union for each datapoint tested on
#sum those and then divide by number of datapoints
#vectorized for speed
def multi_label_accuracy(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    union = real_labels_matrix | predictions_labels_matrix
    #sum(array.T) gets number of 1s in row
    row_wise_accuracy = sum(intersection.T) / sum(union.T)
    return sum(row_wise_accuracy) / real_labels_matrix.shape[0]

#size of intersection of predicted and actual labels divided by size of predicted set for each datapoint tested on
#sum those and divide by number of datapoints
#if no predicted labels, don't count that row towards the precision as that would be undefined
def multi_label_precision(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    precision_sum = 0
    num_rows = 0
    for row in range(intersection.shape[0]):
        if sum(predictions_labels_matrix[row]) > 0: #if there is at least one prediction for this row
            num_rows += 1
            precision_sum += sum(intersection[row]) / sum(predictions_labels_matrix[row])
    if num_rows == 0:
        return 0#no labels predicted at all will give us 0 precision as precision makes no sense here
    return precision_sum / num_rows

#size of intersection of predicted and actual labels divided by size of real label set for each datapoint tested on
#sum those and divide by number of datapoints
#all datapoints should have at least 1 real label in this data set
#vectorized for speed
def multi_label_recall(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    #sum(array.T) gets number of 1s in row
    row_wise_recall = sum(intersection.T) / sum(real_labels_matrix.T)
    return sum(row_wise_recall) / real_labels_matrix.shape[0]

#lower is better
def hamming_loss(real_labels_matrix, predictions_labels_matrix):
    return (np.logical_xor(real_labels_matrix, predictions_labels_matrix)).sum()/(real_labels_matrix.shape[0] * real_labels_matrix.shape[1])

import keras.backend as K

#metric for keras for early stopping
#takes in raw labels from kerass (not yet converted to 0 and 1s)
#NOT the same as accuracy, this is total labels correctly identified divided by union of total labels
#this weights rows with more labels higher, where accruacy does not, but this is still a good metric for early stopping
def raw_multi_label_accuracy(y_true, y_pred):
    positives = K.greater_equal(y_pred, 0.5)
    positives = K.cast(positives, K.floatx())
    new_y_pred = positives #+ ((1-positives)*y_pred)
    intersection = y_true * new_y_pred
    union = 1 -((1-y_true)*(1-new_y_pred))
    accuracy = K.sum(intersection) / K.sum(union)
    return accuracy
    

In [23]:
from keras.callbacks import EarlyStopping
#for early stopping only after certain number of epochs. wait until delay epochs until early stopping
class DelayedEarlyStopping(EarlyStopping):
    def __init__(self, monitor, min_delta=0, patience=0, verbose=0, mode='auto', delay = 100):
        super(DelayedEarlyStopping, self).__init__()
        self.delay = delay

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.delay:
            super().on_epoch_end(epoch, logs)

In [35]:
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten, LSTM
from keras.layers.embeddings import Embedding

model_lstm = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=100, trainable=True)
model_lstm.add(e)
model_lstm.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_lstm.add(MaxPooling1D())
model_lstm.add(LSTM(100))
model_lstm.add(Dense(256, activation='relu'))
model_lstm.add(Dense(len(genre_dict), activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
print(model_lstm.summary())
#model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
model_lstm.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 10, delay=250)], epochs=1000, batch_size=100, verbose=2)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 100, 32)           3200000   
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 99, 100)           6500      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 49, 100)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               25856     
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 1542      
Total params: 3,314,298
Trainable params: 3,314,298
Non-trainable params: 0
_________________________________________________________________


Epoch 54/1000
 - 4s - loss: 0.0014 - raw_multi_label_accuracy: 0.9997 - val_loss: 2.4810 - val_raw_multi_label_accuracy: 0.3144
Epoch 55/1000
 - 4s - loss: 0.0012 - raw_multi_label_accuracy: 0.9997 - val_loss: 2.5189 - val_raw_multi_label_accuracy: 0.3172
Epoch 56/1000
 - 5s - loss: 0.0011 - raw_multi_label_accuracy: 0.9997 - val_loss: 2.5526 - val_raw_multi_label_accuracy: 0.3164
Epoch 57/1000
 - 5s - loss: 9.4884e-04 - raw_multi_label_accuracy: 0.9997 - val_loss: 2.5893 - val_raw_multi_label_accuracy: 0.3199
Epoch 58/1000
 - 5s - loss: 8.8635e-04 - raw_multi_label_accuracy: 0.9998 - val_loss: 2.6110 - val_raw_multi_label_accuracy: 0.3153
Epoch 59/1000
 - 5s - loss: 8.2568e-04 - raw_multi_label_accuracy: 0.9997 - val_loss: 2.6532 - val_raw_multi_label_accuracy: 0.3176
Epoch 60/1000
 - 4s - loss: 7.1539e-04 - raw_multi_label_accuracy: 0.9997 - val_loss: 2.6785 - val_raw_multi_label_accuracy: 0.3177
Epoch 61/1000
 - 5s - loss: 6.2930e-04 - raw_multi_label_accuracy: 0.9997 - val_loss: 2.

 - 4s - loss: 4.8222e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.3878 - val_raw_multi_label_accuracy: 0.3291
Epoch 117/1000
 - 4s - loss: 4.7040e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.3951 - val_raw_multi_label_accuracy: 0.3272
Epoch 118/1000
 - 4s - loss: 4.5438e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.4004 - val_raw_multi_label_accuracy: 0.3291
Epoch 119/1000
 - 4s - loss: 4.4334e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.4059 - val_raw_multi_label_accuracy: 0.3272
Epoch 120/1000
 - 4s - loss: 4.3249e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.4118 - val_raw_multi_label_accuracy: 0.3263
Epoch 121/1000
 - 4s - loss: 4.2149e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.4159 - val_raw_multi_label_accuracy: 0.3272
Epoch 122/1000
 - 5s - loss: 4.1034e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.4224 - val_raw_multi_label_accuracy: 0.3263
Epoch 123/1000
 - 5s - loss: 3.9950e-05 - raw_multi_label_accuracy: 1.0000 - val_los

Epoch 178/1000
 - 4s - loss: 1.1946e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.6668 - val_raw_multi_label_accuracy: 0.3337
Epoch 179/1000
 - 4s - loss: 1.1681e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.6682 - val_raw_multi_label_accuracy: 0.3328
Epoch 180/1000
 - 4s - loss: 1.1498e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.6725 - val_raw_multi_label_accuracy: 0.3342
Epoch 181/1000
 - 4s - loss: 1.1283e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.6753 - val_raw_multi_label_accuracy: 0.3328
Epoch 182/1000
 - 4s - loss: 1.1079e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.6791 - val_raw_multi_label_accuracy: 0.3328
Epoch 183/1000
 - 4s - loss: 1.0977e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.6835 - val_raw_multi_label_accuracy: 0.3356
Epoch 184/1000
 - 4s - loss: 1.0643e-05 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.6852 - val_raw_multi_label_accuracy: 0.3337
Epoch 185/1000
 - 4s - loss: 1.0514e-05 - raw_multi_label_accuracy: 1

Epoch 240/1000
 - 4s - loss: 4.1022e-06 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.8352 - val_raw_multi_label_accuracy: 0.3356
Epoch 241/1000
 - 4s - loss: 4.0240e-06 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.8386 - val_raw_multi_label_accuracy: 0.3351
Epoch 242/1000
 - 4s - loss: 3.9596e-06 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.8391 - val_raw_multi_label_accuracy: 0.3351
Epoch 243/1000
 - 4s - loss: 3.9248e-06 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.8418 - val_raw_multi_label_accuracy: 0.3351
Epoch 244/1000
 - 4s - loss: 3.8538e-06 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.8447 - val_raw_multi_label_accuracy: 0.3351
Epoch 245/1000
 - 4s - loss: 3.8090e-06 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.8465 - val_raw_multi_label_accuracy: 0.3351
Epoch 246/1000
 - 4s - loss: 3.7181e-06 - raw_multi_label_accuracy: 1.0000 - val_loss: 3.8490 - val_raw_multi_label_accuracy: 0.3351
Epoch 247/1000
 - 4s - loss: 3.6625e-06 - raw_multi_label_accuracy: 1

In [36]:
def nn_output_to_predictions(res):
    label_predictions = []
    for i in range(res.shape[0]):
        pred = [0]*len(genre_dict)
        for j in range(res.shape[1]):
            if res[i][j] >= .5:
                pred[j] = 1
        label_predictions.append(pred)
    return np.array(label_predictions)

In [38]:
predictions = nn_output_to_predictions(model_lstm.predict(x_test_seq))

In [39]:
y_test[:,0].sum()

189

In [40]:
predictions[:,0].sum()

235

In [41]:
predictions[0]

array([0, 0, 0, 0, 1, 1])

In [42]:
multi_label_accuracy(y_test, predictions)

0.3518807870370373

In [43]:
multi_label_precision(y_test, predictions)

0.4586086956521741

In [44]:
multi_label_recall(y_test, predictions)

0.5240740740740741

In [45]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 64.09143518518519


In [46]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.625
Precision for Action-Adventure: 0.4425531914893617
Recall for Action-Adventure: 0.5502645502645502

Accuruacy for Romance: 0.7239583333333334
Precision for Romance: 0.3135593220338983
Recall for Romance: 0.3217391304347826

Accuruacy for Horror-Thriller: 0.6388888888888888
Precision for Horror-Thriller: 0.46859903381642515
Recall for Horror-Thriller: 0.49743589743589745

Accuruacy for Comedy: 0.5607638888888888
Precision for Comedy: 0.38235294117647056
Recall for Comedy: 0.4619289340101523

Accuruacy for Science Fiction: 0.7395833333333334
Precision for Science Fiction: 0.25510204081632654
Recall for Science Fiction: 0.24509803921568626

Accuruacy for Drama: 0.5572916666666666
Precision for Drama: 0.547752808988764
Recall for Drama: 0.6747404844290658

